In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from tqdm import tqdm


In [ ]:
# Load and preprocess data
def load_and_prepare_data(train_path, val_path):
    train_df = pd.read_csv(train_path).dropna()
    validation_df = pd.read_csv(val_path).dropna()

    train_texts, train_labels = train_df['Word'].tolist(), train_df['Tag'].tolist()
    validation_texts, validation_labels = validation_df['Word'].tolist(), validation_df['Tag'].tolist()

    label_encoder = LabelEncoder()
    train_labels_encoded = label_encoder.fit_transform(train_labels)
    validation_labels_encoded = label_encoder.transform(validation_labels)

    return train_texts, train_labels_encoded, validation_texts, validation_labels_encoded, label_encoder


In [ ]:
# Custom dataset creation
def create_dataset(texts, labels, tokenizer, max_length):
    input_ids_list = []
    attention_mask_list = []
    labels_list = []

    for text, label in zip(texts, labels):
        encoding = tokenizer(
            str(text),
            truncation=True,
            padding='max_length',
            max_length=max_length,
            return_tensors='pt'
        )
        input_ids_list.append(encoding['input_ids'].squeeze())
        attention_mask_list.append(encoding['attention_mask'].squeeze())
        labels_list.append(torch.tensor(label, dtype=torch.long))

    dataset = list(zip(input_ids_list, attention_mask_list, labels_list))
    return dataset

In [ ]:
# Training and evaluation function
def train_and_evaluate(train_texts, train_labels_encoded, validation_texts, validation_labels_encoded, label_encoder, batch_size=8, max_length=128, num_epochs=3):
    bert_model_name = 'bert-base-multilingual-cased'
    tokenizer = BertTokenizer.from_pretrained(bert_model_name)
    model = BertForSequenceClassification.from_pretrained(bert_model_name, num_labels=len(label_encoder.classes_))

    train_dataset = create_dataset(train_texts, train_labels_encoded, tokenizer, max_length)
    validation_dataset = create_dataset(validation_texts, validation_labels_encoded, tokenizer, max_length)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    validation_loader = DataLoader(validation_dataset, batch_size=batch_size)

    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=2e-5)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    print("Start training")
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        correct_train = 0
        total_train = 0

        with tqdm(train_loader, unit="batch") as t:
            for batch in t:
                input_ids, attention_mask, labels = batch
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                logits = outputs.logits

                loss.backward()
                nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
                optimizer.step()

                total_loss += loss.item()
                _, predicted = torch.max(logits, 1)
                total_train += labels.size(0)
                correct_train += (predicted == labels).sum().item()

                t.set_postfix({'loss': total_loss / (t.n + 1), 'accuracy': correct_train / total_train})

    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        with tqdm(validation_loader, unit="batch") as t:
            for batch in t:
                input_ids, attention_mask, labels = batch
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                labels = labels.to(device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                _, predicted = torch.max(logits, 1)

                y_true.extend(labels.cpu().numpy())
                y_pred.extend(predicted.cpu().numpy())

    # Metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    # Return y_true, y_pred, and label encoder for the combined report
    return y_true, y_pred, label_encoder


In [ ]:
# File paths for datasets
datasets = {
      "Tamil": {
        "train": "/content/drive/MyDrive/CURRENT HYBRID AND MBERT/all language correct format dataset /correct_tamil_dataset.csv",
        "validation": "/content/drive/MyDrive/CURRENT HYBRID AND MBERT/all language correct format dataset /correct_tamil_validation"
    },
    "Malayalam": {
        "train": "/content/drive/MyDrive/CURRENT HYBRID AND MBERT/all language correct format dataset /Final_mal_train(80_)  (1).csv",
        "validation": "/content/drive/MyDrive/CURRENT HYBRID AND MBERT/all language correct format dataset /Final_mal_dev(20_) (1).csv"
    },
    "Tulu": {
        "train": "/content/drive/MyDrive/CURRENT HYBRID AND MBERT/all language correct format dataset /correct_tulu_train_set",
        "validation": "/content/drive/MyDrive/CURRENT HYBRID AND MBERT/all language correct format dataset /correct_tulu_validation_set"
    },
    "Kannada": {
        "train": "/content/drive/MyDrive/CURRENT HYBRID AND MBERT/all language correct format dataset /correct_kannada_train",
        "validation": "//content/drive/MyDrive/CURRENT HYBRID AND MBERT/all language correct format dataset /correct_kannada_validation"
    }
}


In [ ]:
# Evaluate the model on different datasets and save reports
for language, paths in datasets.items():
    print(f"Evaluating for {language}")
    train_texts, train_labels_encoded, validation_texts, validation_labels_encoded, label_encoder = load_and_prepare_data(paths['train'], paths['validation'])
    y_true, y_pred, label_encoder = train_and_evaluate(train_texts, train_labels_encoded, validation_texts, validation_labels_encoded, label_encoder)

    # Generate and print classification report for each language
    classification_report_str = classification_report(y_true, y_pred, target_names=label_encoder.classes_)
    print(f"\nClassification Report for {language}:")
    print(classification_report_str)

    # Save the classification report to a text file
    with open(f"{language}_classification_report.txt", "w") as f:
        f.write(f"Classification Report for {language}:\n")
        f.write(classification_report_str)
